In [2]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import json
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
import time
import re

def changes_from_press(stock_data, press_date, period):
    next_day = timedelta(days=1)
    time_after_release = timedelta(days=period)
    

    next_trading_day = press_date
    
    # try to obtain share price at open on press release date
    day_1_price = stock_data[stock_data['Date'] == press_date]['Open'].values


    # increase the date till the first available open price is found
    num_days = 1
    while not day_1_price:
        if num_days > 30:
            return None
        
        next_trading_day = next_trading_day + next_day

        day_1_price = stock_data[stock_data['Date'] == (next_trading_day)]['Open'].values
        num_days = num_days + 1

    # get next day if available
    next_trading_day = next_trading_day + time_after_release
    day_2_price = stock_data[stock_data['Date'] == (next_trading_day)]['Close'].values

    # increase the date till the second available close price is found
    num_days = 1
    while not day_2_price:
        if num_days > 30: 
            return None
        
        next_trading_day = next_trading_day + next_day

        day_2_price = stock_data[stock_data['Date'] == (next_trading_day + next_day)]['Close'].values
        num_days = num_days + 1
    
    # calcualte percent difference between share prices
    pct_change = ((day_2_price - day_1_price) / day_2_price)*100
    pct_change = pct_change[0]
    return pct_change

def get_df(ticker):
    
    stock_data = yf.Ticker(ticker)

    # get historical market data
    stock_hist = stock_data.history(period="max")

    stock_hist.reset_index(inplace=True)

    stock_hist['Date'] = pd.to_datetime(stock_hist['Date']).dt.date

    stock_hist['Pct_Close'] = stock_hist['Close'].pct_change()*100

    return stock_hist

NameError: name 'Firefox' is not defined

In [12]:
options=Options()

options.set_preference("network.http.pipelining", True)
options.set_preference("network.http.proxy.pipelining", True)
options.set_preference("network.http.pipelining.maxrequests", 8)
options.set_preference("content.notify.interval", 500000)
options.set_preference("content.notify.ontimer", True)
options.set_preference("content.switch.threshold", 250000)
options.set_preference("browser.cache.memory.capacity", 65536) # Increase the cache capacity.
options.set_preference("browser.startup.homepage", "about:blank")
options.set_preference("reader.parse-on-load.enabled", False) # Disable reader, we won't need that.
options.set_preference("browser.pocket.enabled", False) # Duck pocket too!
options.set_preference("loop.enabled", False)
options.set_preference("browser.chrome.toolbar_style", 1) # Text on Toolbar instead of icons
options.set_preference("browser.display.show_image_placeholders", False) # Don't show thumbnails on not loaded images.
options.set_preference("browser.display.use_document_colors", False) # Don't show document colors.
options.set_preference("browser.display.use_document_fonts", 0) # Don't load document fonts.
options.set_preference("browser.display.use_system_colors", True) # Use system colors.
options.set_preference("browser.formfill.enable", False) # Autofill on forms disabled.
options.set_preference("browser.helperApps.deleteTempFileOnExit", True) # Delete temprorary files.
options.set_preference("browser.shell.checkDefaultBrowser", False)
options.set_preference("browser.startup.homepage", "about:blank")
options.set_preference("browser.startup.page", 0) # blank
options.set_preference("browser.tabs.forceHide", True) # Disable tabs, We won't need that.
options.set_preference("browser.urlbar.autoFill", False) # Disable autofill on URL bar.
options.set_preference("browser.urlbar.autocomplete.enabled", False) # Disable autocomplete on URL bar.
options.set_preference("browser.urlbar.showPopup", False) # Disable list of URLs when typing on URL bar.
options.set_preference("browser.urlbar.showSearch", False) # Disable search bar.
options.set_preference("extensions.checkCompatibility", False) # Addon update disabled
options.set_preference("extensions.checkUpdateSecurity", False)
options.set_preference("extensions.update.autoUpdateEnabled", False)
options.set_preference("extensions.update.enabled", False)
options.set_preference("general.startup.browser", False)
options.set_preference("plugin.default_plugin_disabled", False)
options.set_preference("permissions.default.image", 2) # I

driver = webdriver.Firefox(options=options)

driver.get('https://www.jpmorganchase.com/ir/news')

data = []

ticker = 'jpm'

# select by value 

stock_hist = get_df(ticker)
time.sleep(1)


driver.find_element(By.CLASS_NAME, "view-more-button").click()



html = driver.page_source


soup = BeautifulSoup(html) 

driver.close()

articles = soup.find_all('div',attrs={'class':'item'})

for article in articles:
    
    try:
        date = article.find('p', attrs={'data-dg-test':'date'}).text
    except:
        # jpm has extra rows that contain no data
        break
        
    title = article.find('p', attrs={'class':'dg-role press-release'}).text.lstrip()


    date = date.lstrip().rstrip()    # remove starting and trailing whitespaces
    title = title.lstrip().rstrip() 
    title = re.sub("\s\s+", " ", title)

    date = datetime.strptime(date, '%B %d, %Y').date()
    pct_change = changes_from_press(stock_hist, date, 1)

    data.append([date, title, pct_change])


data = pd.DataFrame(data, columns=['date', 'press title', '1d change'])

data = data.dropna()

dates_str = [date.strftime("%m/%d/%Y") for date in data['date'].tolist()]
result = data.to_json(orient="values", index=True)


dict_data = {
    'ticker': ticker,
    'data': json.loads(result),
    },


with open(f'./data/{ticker}.json', 'w', encoding='utf-8') as f:
    json.dump(dict_data, f, ensure_ascii=False, indent=4)

C:\Users\AidanLaptop\AppData\Local\Temp\ipykernel_13924\2867264320.py:43: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  while not day_2_price:
C:\Users\AidanLaptop\AppData\Local\Temp\ipykernel_13924\2867264320.py:28: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  while not day_1_price:
